# Benchmarks

## Init API

In [18]:
import wandb
api = wandb.Api()
config_base = 'configs/det_fpn'

## Choose and download checkpoint

In [1]:
artifact_b2 = api.artifact('nkoch-aitastic/van-detection/run_3ijy5bjy_model:v9', type='model')

chkpt = artifact_b2.download()
config = f'{config_base}/fcos_van_b2_fpn_coco.py'

wandb: Downloading large artifact run_3ijy5bjy_model:v9, 391.94MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.0


In [24]:
artifact_resnet_fcos = api.artifact('nkoch-aitastic/van-detection/run_3ouwht0k_model:v0', type='model')

chkpt = f'{artifact_resnet_fcos.download()}/epoch_12.pth'
config = f'configs/refs/fcos_r50_caffe_fpn_gn-head_1x_coco_adam.py'

wandb: Downloading large artifact run_3ouwht0k_model:v0, 367.90MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [ ]:
artifact = api.artifact('nkoch-aitastic/van-detection/run_2gigg6ev_model:v9', type='model')

path = artifact.download()
chkpt = f'{path}/iter_40000.pth'
config = f'{config_base}/fcos_van_b0_fpn_coco.py'

wandb: Downloading large artifact run_2gigg6ev_model:v9, 135.91MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [65]:
artifact = api.artifact('nkoch-aitastic/van-detection/run_36ttdhnx_model:v0', type='model')

path = artifact.download()
chkpt = f'{path}/epoch_12.pth'
config = f'{config_base}/fcos_van_b0_fpn_coco.py'

wandb: Downloading large artifact run_36ttdhnx_model:v0, 135.91MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


## Benchmark FPS

In [66]:
!torchrun --rdzv_backend=c10d --rdzv_endpoint=localhost:29507 mmdetection/tools/analysis_tools/benchmark.py {config} {chkpt} --launcher pytorch

loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
load checkpoint from local path: ./artifacts/run_36ttdhnx_model:v0/epoch_12.pth
/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Done image [50 / 2000], fps: 6.7 img / s, times per image: 149.7 ms / img
Done image [100/ 2000], fps: 10.2 img / s, times per image: 98.5 ms / img
Done image [150/ 2000], fps: 11.2 img / s, times per image: 89.0 ms / img
Done image [200/ 2000], fps: 12.7 img / s, times per image: 78.6 ms / img
Done image [250/ 2000], fps: 13.4 img / s, times per image: 74.9 ms / img
Done image [300/ 2000], fps: 13.9 img / s, times per image: 72.1 ms / img
Done image [350/ 2000], fps: 14.6 img / s, times per i

## Calculate FLOPs for misc configs

In [70]:
%env CUDA_VISIBLE_DEVICES=1
input_shape = "1333 800"
ref_base = 'configs/refs'
refs = [
    'fcos_r50_caffe_fpn_gn-head_1x_coco_adam.py',
    'deformable_detr_r50_16x2_50e_coco.py',
]
configs = [
    'fcos_van_b0_fpn_coco.py',
    'fcos_van_b0_fpn_dcn_1x_coco.py',
    'fcos_van_b0_fpn_dcn_1x_coco_adam_scp_bof.py',
    'fcos_van_b0_starrelu_fpn_coco.py',
    'fcos_van_b2_fpn_coco.py',
    # 'fcos_van_b3_fpn_coco.py',
    'deformable_detr_van_b0_2x1_12e_coco.py',
]

tail_output = "| tail -n 6 | head -n 5"
# tail_output = ""

for ref in refs:
    print(f'{ref}')
    !python mmdetection/tools/analysis_tools/get_flops.py {ref_base}/{ref} --shape {input_shape} {tail_output}

for conf in configs:
    print(f'\n\n{conf}')
    !python mmdetection/tools/analysis_tools/get_flops.py {config_base}/{conf} --shape {input_shape} {tail_output}

env: CUDA_VISIBLE_DEVICES=1
fcos_r50_caffe_fpn_gn-head_1x_coco_adam.py
Input shape: (3, 1344, 800)
Flops: 210.59 GFLOPs
Params: 32.02 M
deformable_detr_r50_16x2_50e_coco.py
/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/mmcv/cnn/utils/flops_counter.py:537: UserWarning: variables __flops__ or __params__ are already defined for the moduleReLU ptflops can affect your code!
  warnings.warn('variables __flops__ or __params__ are already '
/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/mmdet/models/detectors/detr.py:32: UserWarning: Warning! MultiheadAttention in DETR does not support flops computation! Do not use the results in your papers!
  warnings.warn('Warning! MultiheadAttention in DETR does not '
/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/mmdet/models/utils/positional_encoding.py:81: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the '